In [1]:
import pandas as pd
from itertools import product
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

def create_sub_widget(pdict, style):
    ws = []
    for k in pdict.keys():
        w = widgets.Dropdown(options=pdict[k], description=k, style=style, 
                             layout=widgets.Layout(flex='1 3 auto', width='auto'))
        cbox = widgets.Checkbox(value=False, disabled=False, indent=False, 
                             layout=widgets.Layout(flex='1 1 auto', width='auto'))
        l = widgets.link((cbox, 'value'), (w, 'disabled'))
        ws.append(widgets.HBox([cbox, w], 
                               layout=widgets.Layout(display='inline-flex', flex_flow='row')))
    return ws
    
def create_widget(pdict):
    style = {'description_width': 'initial'}
    ws = create_sub_widget(pdict, style)
    form = widgets.Box(ws, 
                layout=widgets.Layout(display='inline-flex', flex_flow='column', border='solid 2px', 
                                      justify_content='space-between'))
    
    interactive_dict = dict()
    for box, key in zip(ws, pdict.keys()):
        cb, dd = box.children
        interactive_dict[f"cb_{key}"] = cb
        interactive_dict[f"{key}"] = dd
    return form, interactive_dict

def check_avaliable(bool_list):
    return sum(bool_list) == 1

def draw_fig1(ax, df_p, trg_col, wgs):
    n = np.arange(len(df_p[trg_col]))
    ax.bar(n, df_p["test_accuracy(best)"])
    for i, v in enumerate(df_p["test_accuracy(best)"]):
        ax.text(i-0.08, v+0.5, f"{v:.2f} %", fontsize=10, color="black")
    if trg_col == "model":
        title = f"Test Accuracy of model comparison"
        xticklabels = df_p[trg_col] + "\n(params: "+ df_p["parameter_size"].astype("str") + ")"
    else:
        title = f"Test Accuracy of model: {wgs['model']} (params: {df_p['parameter_size'].unique()[0]}) comparison"
        xticklabels = df_p[trg_col]
    ax.set_xticks(n)
    ax.set_xticklabels(xticklabels, fontsize=10)
    ax.set_xlabel(f"{trg_col}", fontsize=12)
    ax.set_ylabel("test_accuracy(best)", fontsize=12)
    ax.set_title(title, fontsize=14)
    
def draw(**interactive_dict):
    global df
    
    cboxes, wgs = [], []
    for x in interactive_dict.items():
        if "cb" in x[0]:
            cboxes.append(x)
        else:
            wgs.append(x)
    cboxes, wgs = list(map(dict, [cboxes, wgs]))
    
    if check_avaliable(cboxes.values()):
        trg_col = next(x[0].replace("cb_", "") for x in cboxes.items() if x[1] == True)
        not_trg_dict = dict([x for x in wgs.items() if x[0] != trg_col])
        df_p = df.loc[df[not_trg_dict.keys()].isin(not_trg_dict.values()).all(axis=1), :].copy()
        if trg_col == "model":
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            draw_fig1(ax, df_p, trg_col, wgs)
        else:
            gs_kw = dict(width_ratios=[1], height_ratios=[6, 1])
            fig, axes = plt.subplots(2, 1, figsize=(10, 7), gridspec_kw=gs_kw)
            ax1, ax2 = axes
            ax2.axis(False)
            draw_fig1(ax1, df_p, trg_col, wgs)
            d = dict([x for x in wgs.items() if (x[0] != trg_col) & (x[0] != "model")])
            df_summary = df.loc[df[d.keys()].isin(d.values()).all(axis=1), 
                                ("model", trg_col, "test_accuracy(best)")]\
                .set_index(["model", trg_col]).unstack(-1)
            sort_asc = False if trg_col in ["learning_rate", "optimizer"] else True
            df_summary = df_summary.loc[:, df_summary.columns.sortlevel(level=1, ascending=sort_asc)[0]]
            ax2.set_title(f"Summary Table for `{trg_col}`", fontsize=14)
            ax2.table(cellText=df_summary.values, colWidths = [0.35]*len(df_summary.columns),
                      rowLabels=df_summary.index,
                      colLabels=[f"{trg_col}: {x[1]}" for x in df_summary.columns],
                      cellLoc = 'center', rowLoc = 'center',
                      loc="center")
        plt.tight_layout()
        plt.show()
    else:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        ax.axis(False)
        ax.text(0.15, 0.5, "not avaliable, check only one of the box to fix variable", fontsize=16)
        plt.show()
        
def show(pdict):
    form, interactive_dict = create_widget(pdict)
    out = widgets.interactive_output(draw, interactive_dict)        
    display(form, out)
    
df = pd.read_csv("https://raw.githubusercontent.com/simonjisu/voila-example/master/result.csv",
                 encoding="utf-8")

exp_models = ["linear", "nn", "cnn"]
exp_batchsizes = [32, 128, 256]
exp_epochs = [5, 10, 15]
exp_learningrates = [0.1, 0.01, 0.001]
exp_optimizers = ["GradientDescent", "Adam"]

pdict = {
    "model": exp_models, 
    "batch_size": exp_batchsizes, 
    "epochs": exp_epochs, 
    "learning_rate": exp_learningrates, 
    "optimizer": exp_optimizers
}

# Test Report for model Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/simonjisu/voila-example/blob/master/02_Test%20Report%20for%20model%20Analysis.ipynb)

* 3 types of model: 
    1. simple linear layer + softmax
    2. 2 layer fully-connected neural network(# of hiddens: 1024, 512)
    3. 3 layer convolutional neural network
    
You can fix one of parameters to see comparison by checking parameter box 

For example, if you want to see what is difference between models, 

- [x] model
- [ ] batch_size
- [ ] epochs
- [ ] learning_rate
- [ ] optimizer

In [2]:
show(pdict)

Box(children=(HBox(children=(Checkbox(value=False, indent=False, layout=Layout(flex='1 1 auto', width='auto'))…

Output()